In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import scipy.signal as sig


In [ ]:
Sr_i = 625
Sr_o = 48
vl = Sr_i * 48
vl

In [ ]:
# sample input signal -- at 10 kHz 
f_i = 10
w_i = 2 * np.pi * f_i
t = np.arange(vl)
s_i = np.exp(t * 1j * w_i / Sr_i)

In [ ]:
plt.plot(np.abs(np.fft.fft(s_i)/30000)[0:3000])

In [ ]:
def makeLPF(cutoff, rate, length, taps):
    fdi = np.zeros(length) * 1j
    co_idx = int(length * cutoff / float(rate))
    print(co_idx)
    fdi[0:co_idx] = 1 + 0j
    fdi[co_idx:co_idx+20] = np.linspace(1, 0, 20)
    fdi[int(length/2):] = np.flip(fdi[0:int(length/2)])
    
    tdi = np.fft.ifft(fdi)
    #plt.plot(tdi[0:200], label='pos')
    #plt.plot(tdi[-200:], label='neg')
    #print(tdi[-10:])
    # window it
    win = sig.windows.blackman(taps, sym=False)
    #win = np.ones(taps)
    tidx = int(taps/2)
    #plt.plot(win)
    
    tdi[tidx:taps] = tdi[0:tidx+1]
    tdi[0:tidx] = np.flip(tdi[1:tidx+1])
    plt.plot(tdi, label='prewin')
    tdi[0:taps] = tdi[0:taps]*win
    plt.plot(tdi, label='shift')
    
    tdi[taps:] = 0 + 0j
    tdi = np.real(tdi)
    fo = np.fft.fft(tdi)

    return fo, tdi, fdi
    

In [ ]:
fo2, tdi2, fdi2 = makeLPF(44, 625, vl, 51)
#plt.plot(sig.windows.hamming(41, sym=True))
#plt.plot(tdi[0:160])
#plt.plot(tdi2[0:160])
plt.xlim(20,30)
#plt.ylim(-0.025,0.025)
#plt.plot(tdi2[-160:])
plt.grid()
plt.legend()
tdi2[22:28]

In [ ]:
plt.plot(20 * np.log10(np.abs(np.fft.fftshift(fo)))) #(fo[-5000:])))
plt.plot(20 * np.log10(np.abs(np.fft.fftshift(fo2))))
plt.plot(20 * np.log10(np.abs(np.fft.fftshift(fdi2 + 1e-8))))
plt.grid()
plt.xlim(10000,20000)
plt.ylim(-10,5)
fo.shape

In [ ]:
plt.plot(np.angle(np.fft.fftshift(fo2)))
plt.plot(np.log10(np.abs(np.fft.fftshift(fo2))))
plt.xlim(10000,20000)

In [ ]:
fo51,td51, fi51 = makeLPF(25, 100, vl, 51)
fo41,td41, fi41 = makeLPF(25, 100, vl, 41)

In [ ]:
plt.plot(td51)
plt.plot(td41)
plt.xlim(0,51)
plt.grid()

In [ ]:
plt.plot(20 * np.log10(np.abs(np.fft.fftshift(fo51))))
plt.plot(20 * np.log10(np.abs(np.fft.fftshift(fo41))))
#plt.plot(20 * np.log10(np.abs(np.fft.fftshift(fi41 + 1e-9))))
#plt.ylim(-70,5)
plt.grid()

In [ ]:
def sweep(filt, N):
    slen = filt.shape[0]
    winc = np.linspace(-np.pi, np.pi, N)
    t = np.arange(slen)
    res = np.zeros(N)
    for i in range(N):
        s = np.exp(t * winc[i] * 1j)
        fs = np.fft.fft(s)
        ffs = fs * filt
        r = np.fft.ifft(ffs)
        res[i] = np.sum(np.abs(r)) / N
    return winc, res   
    
    

In [ ]:
theta, res = sweep(fo51, 200)
plt.plot(theta, 20 * np.log10(res))
plt.grid()
plt.ylim(30,50)

In [ ]:
def reSample(v_in, sr_i, sr_o, lpf):
    pfv_i = np.fft.fft(v_in)
    fv_i = pfv_i * lpf
    plt.plot(np.log10(np.abs(pfv_i))[0:5000])
    plt.plot(np.log10(np.abs(fv_i))[0:5000])
    num_out_samps = int(v_in.shape[0] * sr_o / sr_i)
    fv_o = np.zeros(num_out_samps) * 1j
    nos2 = int(num_out_samps / 2)
    fv_o[0:nos2] = fv_i[0:nos2]
    fv_o[nos2:] = fv_i[-nos2:]
    v_o = np.fft.ifft(fv_o)
    # now correct the amplitude
    v_o = v_o * sr_o / float(sr_i)
    return v_o

In [ ]:
s_o = reSample(s_i, Sr_i, Sr_o, fo )

In [ ]:
plt.plot(s_o[0:100])
s_o.shape, s_i.shape[0] * 48 / 625

In [ ]:
plt.plot(s_o[0:48])

In [ ]:
idx=np.arange(1000)
plt.plot(idx, np.log10(np.abs(np.fft.fft(s_o)[0:1000])))
plt.plot(idx+1,np.log10(np.abs(np.fft.fft(s_i)[0:1000])))
plt.grid()
plt.ylim(-2,5)

In [ ]:
def TDResampler(v_in, interp, decim, lpf):

    # interpolate by interp and decimate by decim
    inlen = v_in.shape[0]
    outlen = int(inlen * interp / decim)
    out = np.zeros(outlen) * 1j
    m = 0
    n = lpf.shape[0]
    k = 0
    L = interp
    M = decim
    czero = 0 * (1 + 1j)
    taps = lpf.shape[0]
    for m in range(outlen):
        if n >= v_in.shape[0]:
            break;
        for i in range(taps):
            rsum = lpf[(i * L + k) % taps] * v_in[n - i]
        out[m] = rsum
        n = n + 1
        k = k + M
        while k >= L:
            k = k - L
            n = n + 1
            
    return out



In [ ]:
s_o = TDResampler(s_i, 5, 3, tdi )
Sr_i, Sr_o, s_i.shape

In [ ]:
plt.plot(0.5e6 * s_o[0:200])
plt.plot(s_i[0:200])

In [ ]:
def FDResampler(v_in, filt, U, D):
    # V_in must be of a length so that U/D
    # upsample -- FFT, filter, then stuff. 
    # downsample -- FFT, filter, then trim
    fv_in = np.fft.fft(v_in)
    # filter
    fv_in = fv_in * filt

    # upsample -- absurdly naive
    v_in_len = v_in.shape[0]
    
    # stuff amount is 
    stuff_len = int(v_in_len * U / D)
    # is it really even? 
    if stuff_len * D != v_in_len * U:
        print("Ooops! v_in len %d * %d / %d is not an inter: %f" % (v_in_len, U, D, v_in_len * U / D))
    fv_out = np.zeros(stuff_len) * 1j
    if(stuff_len < v_in_len): 
        sl_h = int(stuff_len / 2)
        fv_out[0:sl_h] = fv_in[0:sl_h]
        fv_out[-sl_h:] = fv_in[-sl_h:]
    else:    
        vilh = int( v_in_len / 2)
        fv_out[0:vilh] = fv_in[0:vilh]
        fv_out[-vilh:] = fv_in[-vilh:]
    # now return result
    return np.fft.ifft(fv_out)
    

In [ ]:
plt.plot(s_i[0:625])

In [ ]:
su2_i = FDResampler(s_i, np.ones(s_i.shape[0]), 48, 625)
su2_i.shape, s_i.shape

In [ ]:
plt.plot(np.real(su2_i[0:48]))
plt.grid()

In [ ]:
### def isInt(a, u, l):
    v = int(a * u / l)
    return a * u == v * l

def nextP235(a):
    best = 10000000
    for e2 in range(20):
        for e3 in range(5):
            for e5 in range(4):
                s = (2 ** e2)  * (3 ** e3) * (5 ** e5)
                if s > (a + 41):
                    if s < best:
                        best = s
    return best

def findGood(Rr, Ra): 
    Sa_min = int(0.05 * Ra)
    Sa_max = int(0.15 * Ra)
    print("Sa_min %d  Sa_max %d" % (Sa_min, Sa_max))
    Sb_min = 1000000
    Sa_opt = 0
    for s in range(Sa_min, Sa_max):
        if isInt(s, Rr, Ra):
            Sr = s * Rr / Ra
            Sb = nextP235(Sr)
            #print("Sr %d Sb %d" % (Sr, Sb))
            if Sb < Sb_min:
                Sb_min = Sb
                Sa_opt = Sr
    return Sb_min, Sa_opt, Sa_opt * Ra / Rr

In [ ]:
findGood(625000, 44100)

In [ ]:
math.factor(32000)

In [ ]:
32000 / 256

In [ ]:
2304 / 48000
